In [1]:
import ipyvuetify as v

In [2]:
## DEFINE IMPORTS ##
import ipywidgets as widgets
import numpy as np
from bqplot import pyplot as plt
import random

import bqplot as bq


from ipywidgets import HBox, VBox, IntSlider, Play, jslink

In [3]:
## DEFINE CONSTANTS  
n_cells = 100      # Number of cells/grid spaces in the 1-D space
    
p_hit = 0.6        # Probility of certainty after a Hit mesurement
p_miss = 0.2       # Probility of certainty after a Miss mesurement
p_exact = 0.8      # Probility of certainty after moving a desired space forward/backwards
p_undershoot = 0.1 # Probility of undershooting (i.e. -1 spaces) after moving desired spaces
p_overshoot = 0.1  # Probility of opershooting (i.e. +1 spaces) after moving desired spaces

## INITIAL SETUP ##

p = n_cells * [1/n_cells] # Uniform probability distribution at the beginning

world = [1 if random.random() > 0.8 else 0 for _ in range(n_cells)] # creating doors for the sensor model
colors = ["white", "black"] # giving the doors black and open spaces white
levels = [0, 1, 2] # ???
# set colors and normalization
# cmap, norm = from_levels_and_colors(levels, colors) # ???



In [4]:
## FUNCTIONS ##

def sense(p, Z):  
    '''
    Takes probility array and World map and returns an updated p with new probililities.  
    '''
    q = []    
    for i in range(n_cells):    
        hit = (world[i] == Z)    
        q.append(p[i]*(hit*p_hit + (1 - hit)*p_miss))    
    norm = sum(q)                                                                                    
    q = [q[i]/norm for i in range(n_cells)]                                                          
    return q    

def move(p, U):    
    ''' 
    Takes p:probility array and U:move_steps and returns an updated p with new probilieties
    '''
    q = []    
    for i in range(n_cells):
        s = p_exact*p[(i-U)%len(p)]                                                                  
        s += p_overshoot*p[(i-U-1)%len(p)]                                                           
        s += p_undershoot*p[(i-U+1)%len(p)]                                                          
        q.append(s)                                                                                  
    return q  

def init():
    '''
    Function initlizes matplotlib graph
    '''
    prob_ax.set_xlim(0, n_cells-1)
    prob_ax.set_ylim(0, 1)
    world_ax.set_ylim(-2,3)
    world_ax.set_xlabel("Position")
    prob_ax.set_ylabel("Probability")
    prob_ax.set_title("Histogram Localization With Measurements")
    return rects, line

def update(i):
    '''
    Function updates the matplotlib graph 
    '''
    global p
    p = sense(p, world[i%100])
    #print(p)
    p = move(p, 1)
    #print(p)
    for rect, h in zip(rects, p):
        rect.set_height(h)
    line.set_xdata((i+1)%n_cells)
    #input()
    return rects, line


### WIDGET FUNCTIONS ###

def simple_sensing_reset(b):
    # Resets the velocity slider to 0 when the reset button is clicked 
    # Updated to also reset the labels appropriately. - jec 14oct2018
    p_hit_slider.value = 0.6
    p_miss_slider.value = 0.2
    
def simple_moving_reset(b):
    # Resets the velocity slider to 0 when the reset button is clicked 
    # Updated to also reset the labels appropriately. - jec 14oct2018
    p_overshoot_slider.value = 0.2
    p_undershoot_slider.value = 0.2
    p_exact_slider.value = 0.8
    
def simple_update(change):
    '''
    Function updates the matplotlib graph 
    '''
    global p, P_bar_graph, position_dot
    p = sense(p, world[position_slider.value %100])
    # print(p)
    # print("{}, {}".format(change.new, change.old))
    p = move(p, (change.new - change.old))
    #print(p)
    ## Update bar graph ##
    x_cells = [i for i in range(n_cells)]
    P_bar_graph.x = x_cells
    P_bar_graph.y = p
    
    ##  Update position point ##
    position_dot.x = [(position_slider.value)% 100]
    
    with output:
        print(f'{p_hit}')

    
def simple_value_update(change):
    global p_hit, p_miss, p_exact, p_overshoot, p_undershoot
    
    p_hit = p_hit_slider.value
    p_miss = p_miss_slider.value
    p_exact = p_exact_slider.value
    p_overshoot = p_overshoot_slider.value
    p_undershoot = p_undershoot_slider.value
    
    
   


In [10]:
## INTERACTIVE/DISPLAY WIDGETS ##

# Initialize slider size
ControlColWidth = '450px'
slider_width = '300px'
slider_width = '300px'
readout_width = '70px'

# Create slider titles
sensing_control_title = widgets.HTML('<b>Sensing Controls</b>:')
moving_control_title = widgets.HTML('<b>Moving Controls</b>:')
play_control_title = widgets.HTML('<b>Play Controls</b>:')
##
## Create control for selecting Figure
##
        ## Sensing ##
p_hit_slider = widgets.FloatSlider(value=p_hit,
                                    min=0,
                                    max=1,
                                    step=0.1,
                                    disabled=False,
                                    continuous_update=True,
                                    style = {'description_width': 'initial'},
                                    description = 'P Hit Value',
                                    orientation='horizontal',
                                    readout=False,
                                    readout_format='.1f',
                                    layout=widgets.Layout(width=slider_width,
                                                           overflow_x='visible', overflow_y='visible')

                                    )


p_miss_slider = widgets.FloatSlider(value=p_miss,
                                    min=0,
                                    max=1,
                                    step=0.1,
                                    disabled=False,
                                    continuous_update=True,
                                    style = {'description_width': 'initial'},
                                    description = 'P Miss Value',
                                    orientation='horizontal',
                                    readout=False,
                                    readout_format='.1f',
                                    layout=widgets.Layout(width=slider_width,
                                                           overflow_x='visible', overflow_y='visible')
                                    )

        ## Moving ##
p_exact_slider = widgets.FloatSlider(value=p_exact,
                                    min=0,
                                    max=1,
                                    step=0.1,
                                    disabled=False,
                                    continuous_update=True,
                                    style = {'description_width': 'initial'},
                                    description = 'P Exact Value',
                                    orientation='horizontal',
                                    readout=False,
                                    readout_format='.1f',
                                    layout=widgets.Layout(width=slider_width,
                                                           overflow_x='visible', overflow_y='visible')
                                    )
p_undershoot_slider = widgets.FloatSlider(value=p_undershoot,
                                    min=0,
                                    max=1,
                                    step=0.1,
                                    disabled=False,
                                    continuous_update=True,
                                    style = {'description_width': 'initial'},
                                    description = 'P Undershoot Value',
                                    orientation='horizontal',
                                    readout=False,
                                    readout_format='.1f',
                                    layout=widgets.Layout(width=slider_width,
                                                           overflow_x='visible', overflow_y='visible')
                                    )

p_overshoot_slider = widgets.FloatSlider(value=p_overshoot,
                                    min=0,
                                    max=1,
                                    step=0.1,
                                    disabled=False,
                                    continuous_update=True,
                                    style = {'description_width': 'initial'},
                                    description = 'P Overshoot Value',
                                    orientation='horizontal',
                                    readout=False,
                                    readout_format='.1f',
                                    layout=widgets.Layout(width=slider_width,
                                                           overflow_x='visible', overflow_y='visible')
                                    )

# Reset buttons
reset_button_sensing = widgets.Button(description='Reset')
reset_button_moving = widgets.Button(description='Reset')




# Play Button

play = widgets.Play(
    value=0,
    min=0,
    max=100,
    step=1,
    interval=100,
    description="Press play",
    disabled=False
)
position_slider = widgets.IntSlider()
widgets.jslink((play, 'value'), (position_slider, 'value'))
play_widget = widgets.HBox([play, position_slider])
play_widget = widgets.VBox([play_control_title, play_widget])


# Define text boxes for readout
    ## Sensing ##
p_hit_readout = widgets.BoundedFloatText(min=p_hit_slider.min, max=p_hit_slider.max, 
                                         value=p_hit_slider.value, 
                                         layout=widgets.Layout(width=readout_width, 
                                                               overflow_x='visible', 
                                                               overflow_y='visible'))
p_miss_readout = widgets.BoundedFloatText(min=p_miss_slider.min, max=p_miss_slider.max, 
                                         value=p_miss_slider.value,
                                         layout=widgets.Layout(width=readout_width, 
                                                               overflow_x='visible', 
                                                               overflow_y='visible'))
    ## Moving ##
p_exact_readout = widgets.BoundedFloatText(min=p_exact_slider.min, max=p_exact_slider.max, 
                                         value=p_exact_slider.value, 
                                         layout=widgets.Layout(width=readout_width, 
                                                               overflow_x='visible', 
                                                               overflow_y='visible'))

p_undershoot_readout = widgets.BoundedFloatText(min=p_undershoot_slider.min, max=p_undershoot_slider.max, 
                                         value=p_undershoot_slider.value, 
                                         layout=widgets.Layout(width=readout_width, 
                                                               overflow_x='visible', 
                                                               overflow_y='visible'))

p_overshoot_readout = widgets.BoundedFloatText(min=p_overshoot_slider.min, max=p_overshoot_slider.max, 
                                         value=p_overshoot_slider.value, 
                                         layout=widgets.Layout(width=readout_width, 
                                                               overflow_x='visible', 
                                                               overflow_y='visible'))


# Link slider and textboxes
    ## sensing ##
widgets.jslink((p_hit_readout, 'value'), (p_hit_slider,'value'))
widgets.jslink((p_miss_readout, 'value'), (p_miss_slider,'value'))
    ## moving ##
widgets.jslink((p_exact_readout, 'value'), (p_exact_slider,'value'))
widgets.jslink((p_overshoot_readout, 'value'), (p_overshoot_slider,'value'))
widgets.jslink((p_undershoot_readout, 'value'), (p_undershoot_slider,'value'))


# Box the sliders and textboxes
    ## sensing ##
box_hit = widgets.HBox([p_hit_slider, p_hit_readout])
box_miss = widgets.HBox([p_miss_slider, p_miss_readout])


box_exact = widgets.HBox([p_exact_slider, p_exact_readout])
box_overshoot = widgets.HBox([p_overshoot_slider, p_overshoot_readout])
box_undershoot = widgets.HBox([p_undershoot_slider, p_undershoot_readout])


vbox_sensing = widgets.VBox([sensing_control_title,reset_button_sensing,box_hit, box_miss])

vbox_moving = widgets.VBox([moving_control_title,reset_button_moving,box_exact, box_overshoot, box_undershoot])

## widget final layout ##
controls_widget = widgets.VBox([vbox_sensing, vbox_moving, play_widget])



## Output widget ##

output = widgets.Output()

# Create the individual controls for stellar masses
########### SET UP 1-D LOCALIZATION HISTOGRAM PLOT ###############
## using bqplot directly instead of plt API ##

# determine maximum and minimum value of X and Y axes
############## CODE  HERE ####################


# Lables and scales for Axes 
          ## Probobility Bar graph ##
x_bar_graph = bq.LinearScale(min=0, max=100)
y_bar_graph = bq.LinearScale(min=0, max= 1)
ax_x_bar_graph = bq.Axis(label="", scale=x_bar_graph)
ax_y_bar_graph = bq.Axis(label="Probobility", scale=y_bar_graph, orientation='vertical')
          ## World Bar graph ##
worldx_bar_graph = bq.LinearScale(min=0, max=100)
worldy_bar_graph = bq.LinearScale(min=0, max=1)
world_color = bq.ColorScale(scheme='Greys') # change this

worldax_x_bar_graph = bq.Axis(label="Position", scale=worldx_bar_graph, grid_lines='none')
worldax_y_bar_graph = bq.Axis(label="", scale=worldy_bar_graph, orientation='vertical', visible=False, grid_lines='none')
world_color_axis = bq.ColorAxis(scale=world_color)

# set up initial conditions
############## CODE HERE ####################

##
## Set up the bar graph
##

    ## Probobility Bar graph ##
x_cells = [i for i in range(n_cells)]
P_bar_graph = bq.Bars(x=x_cells , y=p , scales={'x': x_bar_graph, 'y': y_bar_graph},
                    padding=0.5,
                    align='right',
                    labels=['Initial Sample'])


    ## World Heatmap ##
## get 2d numpy map of world map
color = np.array([world,world])

world_heat_graph = bq.HeatMap(x=np.arange(0,100), y=np.arange(0,2), color=color,
               scales={'x': worldx_bar_graph, 'y': worldy_bar_graph, 'color': world_color})


position_dot = bq.Scatter(x= [1], y=[0.7], scales={'x': worldx_bar_graph, 'y': worldy_bar_graph}, 
                   colors=['red'], stroke='red', default_size=20, fill= True, labels=['Initial Position'])

##
## Construct plot
##
    ## Probability Figure ##
bar_graph_fig = bq.Figure(axes=[ax_x_bar_graph, ax_y_bar_graph],
                          title='Bar Graph',
                           marks=[P_bar_graph],
                         layout={'width': '800px', 'height': ' 300px', 
                             'max_width': '1000px', 'max_height': '500px',
                             'min_width': '600px', 'min_height': '400px', 'border':'none'},
                          fig_margin={'top':50, 'bottom':30, 'left': 60, 'right':30})

    ## World Figure ##
world_bar_graph_fig = bq.Figure(axes=[worldax_x_bar_graph, worldax_y_bar_graph],
                          title='',
                           marks=[world_heat_graph, position_dot],
                         layout={'width': '800px', 'height': '130px', 
                             'max_width': '1200px', 'max_height': '500px',
                             'min_width': '600px', 'min_height': '10px', 'border':'none'},
                                min_aspect_ratio=5, max_aspect_ratio=7, padding_y=0,
                               fig_margin={'top':10, 'bottom':100, 'left': 60, 'right':30})  


# display(bar_graph_fig)
# display(world_bar_graph_fig)

#update Bar Graph
position_slider.observe(simple_update, names=['value'])

    # update sliders #
p_hit_slider.observe(simple_value_update,names=['value'])
p_miss_slider.observe(simple_value_update,names=['value'])
p_exact_slider.observe(simple_value_update,names=['value'])
p_overshoot_slider.observe(simple_value_update,names=['value'])
p_undershoot_slider.observe(simple_value_update,names=['value'])


    # Update Widget
reset_button_sensing.on_click(simple_sensing_reset)
reset_button_moving.on_click(simple_moving_reset)



# box for the figures
fig_box = widgets.VBox([bar_graph_fig, world_bar_graph_fig])


display( widgets.HBox([fig_box, controls_widget]), padding=0)

display(output)


## using plt API ##



## size = 100
## np.random.seed(0)

## x_data = range(size)
## y_data = np.random.randn(size)
## y_data_2 = np.random.randn(size)
## y_data_3 = np.cumsum(np.random.randn(size) * 100.)

## plt.figure(title="Bar Chart")
## plt.bar(np.arange(10), np.random.rand(10))
## plt.show()




## Things to Do ##
## Reset bar graph button ##
## randomize world map button ##

Output()

'\nsize = 100\nnp.random.seed(0)\n\nx_data = range(size)\ny_data = np.random.randn(size)\ny_data_2 = np.random.randn(size)\ny_data_3 = np.cumsum(np.random.randn(size) * 100.)\n\nplt.figure(title="Bar Chart")\nplt.bar(np.arange(10), np.random.rand(10))\nplt.show()\n'

In [ ]:
## volia layout ##

